In [30]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import json
import numpy as np

def AreaList(mean):
    ''' Generating a list of tube stations for Dash Dropdown'''
    data = pd.ExcelFile('../data/Tube_and_Bus_Route_Stops.xls')
    if 'Bus' in mean:
        sheet = 'Bus Regions'
    else:
        sheet = 'Tube Regions'
    area_list = pd.read_excel(data, sheet_name=sheet, skiprows=1)
    return area_list


def MapDataframe(mean, start, end):
    # Setting the json for appropriate mean and opening the file
    if 'Bus' in mean:
        json_file = '../data/bus_region_map.json'
    else:
        json_file = '../data/tube_region_map.json'
    with open(json_file) as jsonfile:
        geojson = json.load(jsonfile)

    if start is None:
        start = ''

    if end is None:
        end = ''

    # Create dataframe from dataset
    df = AreaList(mean)[{'Name', 'Number'}].set_index('Name')
    df["Status"] = np.nan
    df.loc[start, 'Status'] = 'Start'
    df.loc[end, 'Status'] = 'End'

    return df, geojson

def CreateBorders(mean, start, end):
    df, geojson = MapDataframe(mean, start, end)
    df = df.reset_index().dropna()

    mapbox_access_token = "pk.eyJ1IjoiYmFkcnVsbXVzdGFmZmEiLCJhIjoiY2ttMzE1cXgzNGJ0dzJ1bnc0Z3hkZnBpbSJ9." \
                          "GEDuGnidtzWvTiXPCGIX4w"

    fig = px.choropleth_mapbox(df, geojson=geojson, locations='Name',
                               color='Status',
                               color_discrete_map={'Start': 'blue', 'End': 'red'},
                               featureidkey="properties.id",
                               opacity=1,
                               labels={'Status': 'Legend'})

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 10},
                      autosize=True,
                      showlegend=False,
                      mapbox=go.layout.Mapbox(
                          dict(accesstoken=mapbox_access_token),
                          zoom=11, center={"lat": 51.5087, "lon": -0.1346},
                          layers=[{
                              'sourcetype': 'geojson',
                              'source': geojson,
                              'type': 'line',
                          }]
                      ),
                      mapbox_style="streets"
                      )

    return fig


def PassNavigationVariables(mean, start, end):
    mean_map = 'bus_map'
    if 'Tube' in mean:
        mean_map = 'tube_map'

    start_number = df.loc[start, 'Number']
    end_number = df.loc[end, 'Number']
    print('Type: {}'.format(mean_map))
    print('Start group:{},\nEnd droup: {}'.format(start_number, end_number))
    return mean_map, start_number, end_number

#fig = CreateBorders('Bus', 'Aldgate', 'Barbican')

#mean_map, start_number, end_number = PassNavigationVariables('Bus', 'Aldgate', 'Barbican')
#
#mean_map, start_number, end_number = PassNavigationVariables('Bus', None, None)
#
# df, json = MapDataframe('Bus', 'Aldgate', 'Barbican')
#
# print(df)

def ConvertNavigationVariables(mean, start, end):
    data = pd.ExcelFile('../data/Tube_and_Bus_Route_Stops.xls')
    sheet = 'Bus Regions'
    mean_map = 'bus_map'
    if 'Tube' in  mean:
        sheet = 'Tube Regions'
        mean_map = 'tube_map'
    df = pd.read_excel(data, sheet_name=sheet, skiprows=1)

    if start is None:
        start = ''
    if end is None:
        end = ''

    # Create dataframe from dataset
    df = df[{'Name', 'Number'}].set_index('Name')
    start_number = df.loc[start, 'Number']
    end_number = df.loc[end, 'Number']

    return mean_map, start_number, end_number

A, B, C = ConvertNavigationVariables('Bus', 'Aldgate', 'Barbican')
E, F, G = ConvertNavigationVariables('Bus', 'None', 'None')

print(A, B, C)
print(E, F, G)

KeyError: 'None'